In [411]:
import pandas as pd
import numpy as np
import altair as alt

In [412]:
crime=pd.read_csv('crime_and_incarceration_by_state.csv')

In [413]:
crime

,jurisdiction,includes_jails,year,prisoner_count,crime_reporting_change,crimes_estimated,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
0,FEDERAL,False,2001,149852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALABAMA,False,2001,24741,False,False,4468912.0,19582.0,379.0,1369.0,NaN,5584.0,12250.0,173253.0,40642.0,119992.0,12619.0
2,ALASKA,True,2001,4570,False,False,633630.0,3735.0,39.0,501.0,NaN,514.0,2681.0,23160.0,3847.0,16695.0,2618.0
3,ARIZONA,False,2001,27710,False,False,5306966.0,28675.0,400.0,1518.0,NaN,8868.0,17889.0,293874.0,54821.0,186850.0,52203.0
4,ARKANSAS,False,2001,11489,False,False,2694698.0,12190.0,148.0,892.0,NaN,2181.0,8969.0,99106.0,22196.0,69590.0,7320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,VIRGINIA,False,2016,29882,False,False,8414380.0,18495.0,482.0,NaN,2830.0,4826.0,10357.0,157292.0,20159.0,127285.0,9848.0
812,WASHINGTON,False,2016,17228,False,False,7280934.0,22101.0,195.0,NaN,3133.0,5649.0,13124.0,254994.0,49249.0,173423.0,32322.0
813,WEST VIRGINIA,False,2016,5899,False,False,1828637.0,6633.0,85.0,NaN,684.0,720.0,5144.0,37282.0,9127.0,25657.0,2498.0
814,WISCONSIN,False,2016,23163,False,False,5772917.0,17716.0,232.0,NaN,2005.0,4707.0,10772.0,111911.0,19498.0,82455.0,9958.0


In [414]:
crime_preprocess1=crime[['jurisdiction','year','prisoner_count','state_population']]

In [415]:
crime_preprocess1['prisoner_rate']=crime_preprocess1['prisoner_count']/crime_preprocess1['state_population']

C:\Users\zyk54\AppData\Local\Temp\ipykernel_14436\2999134976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_preprocess1['prisoner_rate']=crime_preprocess1['prisoner_count']/crime_preprocess1['state_population']


In [416]:
crime_data=crime_preprocess1[(crime_preprocess1['year']>=2010) & (crime_preprocess1['year']<=2016)][crime_preprocess1['jurisdiction']!='FEDERAL']

C:\Users\zyk54\AppData\Local\Temp\ipykernel_14436\1829990137.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  crime_data=crime_preprocess1[(crime_preprocess1['year']>=2010) & (crime_preprocess1['year']<=2016)][crime_preprocess1['jurisdiction']!='FEDERAL']


In [417]:
crime_data.jurisdiction=crime_data.jurisdiction.str.lower().str.title()

In [418]:
import altair as alt
from vega_datasets import data
df = crime_data
state_to_id = {
    'Alabama': '1', 'Alaska': '2', 'Arizona': '4', 'Arkansas': '5', 'California': '6', 'Colorado': '8', 'Connecticut': '9', 'Delaware': '10', 'District of Columbia': '11', 'Florida': '12', 'Georgia': '13', 'Hawaii': '15', 'Idaho': '16', 'Illinois': '17', 'Indiana': '18', 'Iowa': '19', 'Kansas': '20', 'Kentucky': '21', 'Louisiana': '22', 'Maine': '23', 'Maryland': '24', 'Massachusetts': '25', 'Michigan': '26', 'Minnesota': '27', 'Mississippi': '28', 'Missouri': '29', 'Montana': '30', 'Nebraska': '31', 'Nevada': '32', 'New Hampshire': '33', 'New Jersey': '34', 'New Mexico': '35', 'New York': '36', 'North Carolina': '37', 'North Dakota': '38', 'Ohio': '39', 'Oklahoma': '40', 'Oregon': '41', 'Pennsylvania': '42', 'Rhode Island': '44', 'South Carolina': '45', 'South Dakota': '46', 'Tennessee': '47', 'Texas': '48', 'Utah': '49', 'Vermont': '50', 'Virginia': '51', 'Washington': '53', 'West Virginia': '54', 'Wisconsin': '55', 'Wyoming': '56', 'Puerto Rico': '72'
}
df['id'] = df['jurisdiction'].apply(lambda x: state_to_id[x])
states = alt.topo_feature(data.us_10m.url, 'states')

base_map = alt.Chart(states).mark_geoshape(
    stroke='white',
    strokeWidth=1
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df[df['year']==2016], 'id', ['jurisdiction', 'prisoner_rate'])
).properties(
    width=700,
    height=400,
    title="US State Prisoner Rates"
).project(
    type='albersUsa'
)

choropleth = alt.Chart(states).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color('prisoner_rate:Q', scale=alt.Scale(scheme='reds'), legend=alt.Legend(title="Prisoner Rate")),
    tooltip=[alt.Tooltip('jurisdiction:N', title='State'), alt.Tooltip('prisoner_rate:Q', title='Prisoner Rate', format='.4f')]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['jurisdiction', 'prisoner_rate'])
)

click = alt.selection_multi(fields=['jurisdiction'])

line_chart = alt.Chart(df).mark_line().encode(
    x='year:N',
    y='prisoner_rate:Q',
    color='jurisdiction:N'
).transform_filter(
    click
)

layered_chart = (base_map + choropleth).add_selection(click) | line_chart

layered_chart


c:\Users\zyk54\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\zyk54\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)

In [419]:
education=pd.read_csv('ACSST5Y2016.S1501-Data.csv')

In [420]:
education_df=education[['Geographic Area Name','Population 18 to 24 years','Less than high school graduate','High school graduate (includes equivalency)','Some college or associate\'s degree','Bachelor\'s degree or higher']]

In [421]:
education_df

,Geographic Area Name,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher
0,Alabama,474567,74167,146337,219193,34870
1,Alaska,78540,11687,30608,31234,5011
2,Arizona,668324,113051,209298,296547,49428
3,Arkansas,287647,39588,93949,133025,21085
4,California,3959511,519677,1149057,1913075,377702
5,Colorado,522764,71670,150050,242768,58276
6,Connecticut,348789,39536,99876,159872,49505
7,Delaware,89588,13455,29120,37660,9353
8,District of Columbia,81327,8084,20021,34337,18885
9,Florida,1774488,283726,535190,807487,148085


In [422]:
education_df['Less than high school graduate']=education_df['Less than high school graduate']/education_df['Population 18 to 24 years']
education_df['High school graduate (includes equivalency)']=education_df['High school graduate (includes equivalency)']/education_df['Population 18 to 24 years']
education_df['Some college or associate\'s degree']=education_df['Some college or associate\'s degree']/education_df['Population 18 to 24 years']
education_df['Bachelor\'s degree or higher']=education_df['Bachelor\'s degree or higher']/education_df['Population 18 to 24 years']

C:\Users\zyk54\AppData\Local\Temp\ipykernel_14436\2121983858.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  education_df['Less than high school graduate']=education_df['Less than high school graduate']/education_df['Population 18 to 24 years']
C:\Users\zyk54\AppData\Local\Temp\ipykernel_14436\2121983858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  education_df['High school graduate (includes equivalency)']=education_df['High school graduate (includes equivalency)']/education_df['Population 18 to

In [423]:
education_df

,Geographic Area Name,Population 18 to 24 years,Less than high school graduate,High school graduate (includes equivalency),Some college or associate's degree,Bachelor's degree or higher
0,Alabama,474567,0.156284,0.308359,0.461880,0.073478
1,Alaska,78540,0.148803,0.389712,0.397683,0.063802
2,Arizona,668324,0.169156,0.313168,0.443717,0.073958
3,Arkansas,287647,0.137627,0.326612,0.462459,0.073302
4,California,3959511,0.131248,0.290202,0.483159,0.095391
5,Colorado,522764,0.137098,0.287032,0.464393,0.111477
6,Connecticut,348789,0.113352,0.286351,0.458363,0.141934
7,Delaware,89588,0.150188,0.325044,0.420369,0.104400
8,District of Columbia,81327,0.099401,0.246179,0.422209,0.232211
9,Florida,1774488,0.159892,0.301602,0.455054,0.083452


In [424]:
melted_df = pd.melt(education_df, id_vars=['Geographic Area Name'], 
                    value_vars=["Less than high school graduate", 
                                "High school graduate (includes equivalency)", 
                                "Some college or associate's degree", 
                                "Bachelor's degree or higher"],
                    var_name='Education Level', value_name='Percentage')

In [425]:
melted_df

,Geographic Area Name,Education Level,Percentage
0,Alabama,Less than high school graduate,0.156284
1,Alaska,Less than high school graduate,0.148803
2,Arizona,Less than high school graduate,0.169156
3,Arkansas,Less than high school graduate,0.137627
4,California,Less than high school graduate,0.131248
...,...,...,...
203,Washington,Bachelor's degree or higher,0.105964
204,West Virginia,Bachelor's degree or higher,0.077563
205,Wisconsin,Bachelor's degree or higher,0.105616
206,Wyoming,Bachelor's degree or higher,0.062112


In [426]:
df=df.merge(melted_df, left_on='jurisdiction', right_on='Geographic Area Name', how='left')

In [427]:
df

,jurisdiction,year,prisoner_count,state_population,prisoner_rate,id,Geographic Area Name,Education Level,Percentage
0,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,Less than high school graduate,0.156284
1,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,High school graduate (includes equivalency),0.308359
2,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,Some college or associate's degree,0.461880
3,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,Bachelor's degree or higher,0.073478
4,Alaska,2010,5369,714146.0,0.007518,2,Alaska,Less than high school graduate,0.148803
...,...,...,...,...,...,...,...,...,...
1395,Wisconsin,2016,23163,5772917.0,0.004012,55,Wisconsin,Bachelor's degree or higher,0.105616
1396,Wyoming,2016,2352,584910.0,0.004021,56,Wyoming,Less than high school graduate,0.143935
1397,Wyoming,2016,2352,584910.0,0.004021,56,Wyoming,High school graduate (includes equivalency),0.320186
1398,Wyoming,2016,2352,584910.0,0.004021,56,Wyoming,Some college or associate's degree,0.473767


In [428]:
df

,jurisdiction,year,prisoner_count,state_population,prisoner_rate,id,Geographic Area Name,Education Level,Percentage
0,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,Less than high school graduate,0.156284
1,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,High school graduate (includes equivalency),0.308359
2,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,Some college or associate's degree,0.461880
3,Alabama,2010,27345,4785401.0,0.005714,1,Alabama,Bachelor's degree or higher,0.073478
4,Alaska,2010,5369,714146.0,0.007518,2,Alaska,Less than high school graduate,0.148803
...,...,...,...,...,...,...,...,...,...
1395,Wisconsin,2016,23163,5772917.0,0.004012,55,Wisconsin,Bachelor's degree or higher,0.105616
1396,Wyoming,2016,2352,584910.0,0.004021,56,Wyoming,Less than high school graduate,0.143935
1397,Wyoming,2016,2352,584910.0,0.004021,56,Wyoming,High school graduate (includes equivalency),0.320186
1398,Wyoming,2016,2352,584910.0,0.004021,56,Wyoming,Some college or associate's degree,0.473767


In [429]:
import altair as alt
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, 'states')
initial_filter = alt.condition(click, alt.value(1), alt.value(0))
base_map = alt.Chart(states).mark_geoshape(
    stroke='white',
    strokeWidth=1
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df[df['year']==2016], 'id', ['jurisdiction', 'prisoner_rate'])
).properties(
    width=700,
    height=500,
    title="US State Prisoner Rates In 2016"
).project(
    type='albersUsa'
)

choropleth = alt.Chart(states).mark_geoshape(
    stroke='white'
).encode(
    color=alt.Color('prisoner_rate:Q', scale=alt.Scale(scheme='reds'), legend=alt.Legend(title="Prisoner Rate")),
    tooltip=[alt.Tooltip('jurisdiction:N', title='State'), alt.Tooltip('prisoner_rate:Q', title='Prisoner Rate', format='.4f')]
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['jurisdiction', 'prisoner_rate'])
)

click = alt.selection_point(fields=['jurisdiction'])

pie_chart=alt.Chart(df).transform_filter(
    alt.datum.year==2016
).mark_arc().encode(
    theta=alt.Theta(field="Percentage", type="quantitative"),
    color=alt.Color(field="Education Level", type="nominal"),
    tooltip=['Education Level', 'Percentage']
).transform_filter(
    click
).properties(
    title='Education Distribution of selected states in 2016'
)

layered_chart = (base_map + choropleth).add_params(click) | (pie_chart)

layered_chart

alt.HConcatChart(...)

: 